import libraries

In [22]:
import os
import json
import numpy as np
import pandas as pd
from utils import load_json, load_pickle

Settings

In [46]:
baseline_folder = "/data/rbg/users/klingmin/projects/ML_MS_analysis/cached_results/FP_prediction/baselines/models_cached"
MIST_folder = "../FP_prediction/mist/best_models/"

To consolidate the best performance for each setting

In [155]:
# Manually add in (hack)

all_folders = []
all_test_performances = {"loss": {}, "jaccard": {}} 

for folder in [baseline_folder]:

    for model_type in os.listdir(folder):
        subfolder = os.path.join(folder, model_type)
        for dataset in os.listdir(subfolder):
            subsubfolder = os.path.join(subfolder, dataset)
            for checkpoint in os.listdir(subsubfolder):
                if "wo_aux" in checkpoint: continue
                if "new_stuff" in subsubfolder: continue
                all_folders.append(os.path.join(subsubfolder, checkpoint))


for folder in [MIST_folder]:

    for dataset in os.listdir(folder):
        subsubfolder = os.path.join(folder, dataset)
        for checkpoint in os.listdir(subsubfolder):
            if "wo_aux" in checkpoint: continue
            all_folders.append(os.path.join(subsubfolder, checkpoint))


for f in all_folders:

    try:
        expt_name = f.split("/")[-1]
        test_performance = load_json(os.path.join(f, "test_performance.json"))
        for k, v in test_performance.items():
            if k == "threshold": continue 
            all_test_performances[k][expt_name] = round(v, 5)
    except Exception as e:
        print(e)
        continue


Format the table

In [156]:
results = pd.DataFrame.from_dict(all_test_performances)
results = results.reset_index()
results = results.rename(columns = {"index": "expt_name"})
results.loc[:, "dataset"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[0]) 
results.loc[:, "meta"] = results.loc[:, "expt_name"].apply(lambda x: "meta" in x) 
results.loc[:, "model"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[1]) 
results.loc[:, "split"] = results.loc[:, "expt_name"].apply(lambda x: "_".join(x.split("_")[-2:]).replace("4096_", "")) 

results = results.loc[:, ["dataset", "model", "split", "meta", "loss", "jaccard"]]

results = results.sort_values(by = 'split', ascending = False)
results = results.sort_values(by = 'model', ascending = False)
results = results.sort_values(by = 'dataset', ascending = False)

Identify the best results

In [159]:
results_wo_meta = results.loc[results.loc[:, "meta"].apply(lambda x: x is False), :]
results_w_meta = results.loc[results.loc[:, "meta"].apply(lambda x: x is True), :]

dataset = "NIST2023"
split = "random"
model = "MS"
results.loc[results.apply(lambda x : x["dataset"] == dataset and x["split"] == split and x["model"] in [model, "MIST"], axis = 1), :]

,dataset,model,split,meta,loss,jaccard
54,NIST2023,MIST,random,True,0.01628,0.62107


Consolidate best results in a table

In [152]:
best_baselines_folder = "../FP_prediction/baseline_models/best_models/"
best_MIST_folder = "../FP_prediction/mist/best_models/"

all_folders = []
all_test_performances = {"loss": {}, "jaccard": {}} 

for folder in [best_baselines_folder, best_MIST_folder]:

    for dataset in os.listdir(folder):
        subfolder = os.path.join(folder, dataset)
        for checkpoint in os.listdir(subfolder):
            all_folders.append(os.path.join(subfolder, checkpoint))

for f in all_folders:

    try:
        expt_name = f.split("/")[-1]
        test_performance = load_json(os.path.join(f, "test_performance.json"))
        for k, v in test_performance.items():
            if k == "threshold": continue 
            all_test_performances[k][expt_name] = round(v, 5)
    except Exception as e:
        print(e)
        continue


In [153]:
results = pd.DataFrame.from_dict(all_test_performances)
results = results.reset_index()
results = results.rename(columns = {"index": "expt_name"})
results.loc[:, "dataset"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[0]) 
results.loc[:, "meta"] = results.loc[:, "expt_name"].apply(lambda x: "meta" in x) 
results.loc[:, "model"] = results.loc[:, "expt_name"].apply(lambda x: x.split("_")[1]) 
results.loc[:, "split"] = results.loc[:, "expt_name"].apply(lambda x: "_".join(x.split("_")[-2:]).replace("4096_", "")) 

results = results.loc[:, ["dataset", "model", "split", "meta", "loss", "jaccard"]]

results = results.sort_values(by = 'split', ascending = False)
results = results.sort_values(by = 'model', ascending = False)
results = results.sort_values(by = 'dataset', ascending = False)


for dataset in ["C", "MSG", "NIST2023"]:

    for split in ["random", "scaffold_vanilla", "inchikey_vanilla"]:

        subresults = results.loc[results.apply(lambda x : x["dataset"] == dataset and x["split"] == split, axis = 1), :]
        print(subresults)
        print()


   dataset    model   split   meta     loss  jaccard
19       C  formula  random   True  0.03671  0.48567
17       C   binned  random   True  0.03929  0.46732
21       C       MS  random   True  0.04029  0.43371
32       C     MIST  random  False  0.02919  0.54347

   dataset    model             split  meta     loss  jaccard
31       C     MIST  scaffold_vanilla  True  0.07181  0.23961
20       C  formula  scaffold_vanilla  True  0.06209  0.23466
18       C       MS  scaffold_vanilla  True  0.06239  0.23231
23       C   binned  scaffold_vanilla  True  0.06392  0.23826

   dataset    model             split   meta     loss  jaccard
30       C     MIST  inchikey_vanilla  False  0.05491  0.31058
15       C  formula  inchikey_vanilla   True  0.05688  0.25098
22       C  formula  inchikey_vanilla  False  0.05826  0.24530
16       C       MS  inchikey_vanilla   True  0.05797  0.24940

   dataset    model   split   meta     loss  jaccard
2      MSG  formula  random   True  0.02318  0.63821
3